# Analyze sample
We'll detect outliers and remove them. Participants who didn't perform the tasks sufficiently well will be excluded from further analyses. It's important to note, however, that for these individuals the task was either to difficult to perform, or the instructions didn't convey the proper execution of the task for these individuals.

In [68]:
# Basic imports and setup.

import sys
import logging
from pathlib import Path

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from neuropsymodelcomparison.dataprocessing import analysis
from neuropsymodelcomparison import plot

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

# Default file format for figures.
pio.kaleido.scope.default_format = "pdf"

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Get preprocessed trials data.

In [69]:
data_path = Path('../data/')
interim_path = data_path / 'interim'
trials_filepath = interim_path / 'trials.csv'
df = pd.read_csv(trials_filepath, index_col='id')
# Easier on memory and faster groupby.
df[['user', 'session', 'block', 'condition', 'task']] = df[['user', 'session', 'block', 'condition', 'task']].astype('category')

## Outlier Detection
Outlier detection by covariance estimation in a Gaussian distributed dataset.

In [70]:
# Detect outliers in whole dataset.
contamination = 0.056  # Proportion of outliers in the data set.

# ToDo: Maybe detect outliers per session since we do the analyses mainly per session?
outliers, z = analysis.get_outlyingness(df[['df1', 'df2']].values, contamination=contamination)
df['outlier'] = outliers.astype(bool)
n_trials_outliers = df['outlier'].value_counts()[True]
# Todo: Exclude if mean sum is not in the vicinity 100-150 after outlier removal?

### Plot Final States with Outlier Threshold

Display point clouds of data for final state values of degrees of freedom 1 and 2, colored by block. The subspace of task goal 1 is presented as a line. The goal for the additional task is represented as a red cross.

In [71]:
fig_trials_scatter = plot.generate_trials_figure(df, contour_data=z)
fig_trials_scatter.show()

## Exclude sessions with insufficient data
If a user didn't perform at least 60% of trials sufficiently well, exclude that user's session from further analyses.

In [72]:
# Minimum number of trials per block for being included in the analyses.
trials_proportion_theshold = 0.6  # proportion of total trials per block.
trials_total_count = 30
trials_count_threshold = int(trials_total_count * trials_proportion_theshold)

# Keep track of how many sessions there are before filtering.
n_sessions = df.groupby(['user', 'session']).ngroups

# Filter outliers
df = df[~df['outlier']]
df.drop(columns=['outlier'], inplace=True)

In [73]:
# Aggregate valid trial counts.
df_counts = df.groupby(['user', 'session', 'condition']).apply(lambda x: x['block'].value_counts()).reset_index().sort_values('level_3').rename({'level_3': 'block', 'block': 'valid trials count'}, axis='columns')

# Display some more information about users.
users = pd.read_csv(data_path / 'raw/users.csv')
df_counts['gender'] = df_counts['user'].map(users['gender'])
df_counts['age_group'] = df_counts['user'].map(users['age_group'])
df_counts['gaming_exp'] = df_counts['user'].map(users['gaming_exp'])

In [74]:
# Bar plot.
fig_exclusions = px.bar(df_counts, x='user', y='valid trials count', color='block', barmode='group', opacity=0.9, hover_data=['condition', 'gender', 'age_group', 'gaming_exp'], width=1000)

# Add threshold.
fig_exclusions.add_trace(go.Scatter(
    x=df_counts['user'],
    y=[trials_count_threshold] * len(df_counts),
    mode='lines',
    name="Inclusion Threshold",
    marker={'color': 'black'},
    opacity=0.8,
    hovertemplate=f"minimum count: {trials_count_threshold}",
))

# Mark excluded.
fig_exclusions.add_trace(go.Scatter(
    x=df_counts.loc[df_counts['valid trials count'] < trials_count_threshold, :]['user'],
    y=[trials_count_threshold] * len(df_counts.loc[df_counts['valid trials count'] < trials_count_threshold, :]),
    mode='markers',
    marker_symbol='x',
    marker_size=15,
    name="Excluded",
    marker={'color': 'black'},
    hovertemplate="Excluded",
))
# Make legend horizontal.
fig_exclusions.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig_exclusions.show()

In [75]:
# Remove sessions with insufficient data from dataframe.
df = df.groupby(['user', 'session']).filter(lambda x: (x['block'].value_counts() >= trials_count_threshold).all())
n_sessions_excluded_by_trials_count = n_sessions - df.groupby(['user', 'session']).ngroups

## Decribe Sample

## Calculate Projection Lengths
We're ultimately interested in the variance of final states in the directions parallel and orthogonal to the uncontrolled manifold. 
We can express the variance of the set $K$ of final state vectors $\{\mathbf{x}_i\}$ in space $R^P$ (with $P==2$) in the direction of vector $\mathbf{v}$ as the variance of the projections of final states on this direction. 
The length of a projection of vector $\mathbf{x}$ on vector $\mathbf{v}$ can be calculated as $\mathbf{\frac{v^T x}{\sqrt{v^T v}}}$.  
Zhang et al. (2008) utilize the Rayleigh fraction for calculating variance of finger force vectors $\mathbf{x}$ in the direction of the uncontrolled manifold vector $\mathbf{v}$: $\mathbf{\frac{v^T Cov(x) v}{v^T v}}$  

The length of projections can also be expressed as the coefficients $a$ and $b$ in a linear combination of new basis vectors: $$\vec{x} - \overline{x} = a\hat{v}_{\parallel UCM} + b\hat{v}_{\perp UCM}$$ with $\vec{x}$ being a 2-dimensional final state of degrees of freedom $\vec{x}=\mathbf{[df_1 \; df_2]}$ and $\overline{x}$ being the mean vector of the set $K$ of final states. $\hat{v}_{\parallel UCM}$ and $\hat{v}_{\perp UCM}$ are unit base vectors ($\|\hat{v}_{\parallel UCM}\|=\|\hat{v}_{\perp UCM}\|=1$).  
In this case we use a simple matrix multiplication in $R^2$ to get the lengths of the projections: $$\mathbf{[a \; b] = (x - \overline{x})A}$$ with $A$'s column vectors being the unit basis vectors parallel and orthogonal to the UCM.

In [76]:
# Calculate projections onto vectors parallel and orthogonal to UCM.
ucm_vec = analysis.get_ucm_vec()
projections = df.groupby(['user', 'session', 'task'])[['df1', 'df2']].apply(analysis.get_projections, ucm_vec)
df = pd.concat([df, projections], axis='columns')

## Save Data

In [77]:
import shutil

# Save final state data with projections.
destination_path = data_path / 'preprocessed'
df.to_csv(destination_path / 'trials.csv')

logging.info(f"Written preprocessed trials data to {(destination_path / 'trials.csv').resolve()}")

# Save data about exclusions. Make a copy and add content, so we can run this notebook separately from the previous one.
sampling_path = data_path / 'interim/sampling.txt'
shutil.copy(str(sampling_path), str(destination_path / sampling_path.name))  # With Python version < 3.9 we need to provide strings.
sampling_path = destination_path / sampling_path.name

with sampling_path.open(mode='a') as f:
    f.write(f"contamination={contamination}\n")
    f.write(f"trials_proportion_theshold={trials_proportion_theshold}\n")
    f.write(f"trials_count_threshold={trials_count_threshold}\n")
    f.write(f"n_trials_outliers={n_trials_outliers}\n")
    f.write(f"n_sessions_excluded_by_trials_count={n_sessions_excluded_by_trials_count}\n")

logging.info(f"Written sampling data to {sampling_path.resolve()}")

# Save figures.
figures_path = Path('../reports/figures')
fig_trials_scatter_filepath = figures_path / 'outliers_scatter.pdf'
fig_trials_scatter.write_image(str(fig_trials_scatter_filepath))
logging.info(f"Written figure to {fig_trials_scatter_filepath.resolve()}")

fig_exclusions_filepath = figures_path / 'exclusions_barplot.pdf'
fig_exclusions.write_image(str(fig_exclusions_filepath))
logging.info(f"Written figure to {fig_exclusions_filepath.resolve()}")


INFO:root:Written preprocessed trials data to /home/olaf/code/NeuroPsyResearchAnalysis/data/preprocessed/trials.csv
INFO:root:Written sampling data to /home/olaf/code/NeuroPsyResearchAnalysis/data/preprocessed/sampling.txt
INFO:root:Written figure to /home/olaf/code/NeuroPsyResearchAnalysis/reports/figures/outliers_scatter.pdf
INFO:root:Written figure to /home/olaf/code/NeuroPsyResearchAnalysis/reports/figures/exclusions_barsplot.pdf
